# Introduction

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

Instructions:

Subset, clean, and reformat the bank_marketing.csv dataset to create and store three new files based on the requirements detailed in the notebook.

 - Split and tidy bank_marketing.csv, storing as three DataFrames called client, campaign, and economics, each containing the columns outlined in the notebook and formatted to      the data types listed.
 - Save the three DataFrames to csv files, without an index, as client.csv, campaign.csv, and economics.csv respectively.

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

In [3]:
df = pd.read_csv("bank_marketing.csv")

# printing to verify data
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


## Part 1 : Reading in and splitting the data

In [65]:
# creating client DF
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

# printing to verify data
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [70]:
# creating campaign DF
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'day', 'month']]

# printing to verify data
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,day,month
0,0,1,261,0,nonexistent,no,13,may
1,1,1,149,0,nonexistent,no,19,may
2,2,1,226,0,nonexistent,no,23,may
3,3,1,151,0,nonexistent,no,27,may
4,4,1,307,0,nonexistent,no,3,may


In [53]:
# creating economics DF
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]

# printing to verify data
economics.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


## Part 2 : Cleaning the data

In [66]:
# making transformations on client DF
client['job'] = client['job'].str.replace('.', '_')
client['education'] = client['education'].str.replace('.', '_')
client.loc[client['education'] == 'unknown', 'education'] = np.NaN
client.loc['education'] = client['education'].replace('unknown', np.NaN)

client.loc[client['credit_default'] != 'yes', 'credit_default'] = False
client.loc[client['credit_default'] == 'yes', 'credit_default'] = True

client.loc[client['mortgage'] != 'yes', 'mortgage'] = False
client.loc[client['mortgage'] == 'yes', 'mortgage'] = True

# printing to verify data
client.head()

C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9136\1077206539.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['job'] = client['job'].str.replace('.', '_')
C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9136\1077206539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].str.replace('.', '_')
C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9136\1077206539.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

,client_id,age,job,marital,education,credit_default,mortgage
0,0.0,56.0,housemaid,married,basic_4y,False,False
1,1.0,57.0,services,married,high_school,False,False
2,2.0,37.0,services,married,high_school,False,True
3,3.0,40.0,admin_,married,basic_6y,False,False
4,4.0,56.0,services,married,high_school,False,False


In [71]:
# making transformations on campaign DF

campaign.loc[campaign['previous_outcome'] != 'success', 'previous_outcome'] = False
campaign.loc[campaign['previous_outcome'] == 'success', 'previous_outcome'] = True

campaign.loc[campaign['campaign_outcome'] != 'yes', 'campaign_outcome'] = False
campaign.loc[campaign['campaign_outcome'] == 'yes', 'campaign_outcome'] = True

campaign['year'] = '2022'
campaign['month'] = campaign['month'].replace(['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])

campaign['day'] = campaign['day'].astype(str)
campaign['last_contact_date'] = campaign['year']+'-'+campaign['month']+'-'+campaign['day']
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'])

campaign.drop(['month', 'year', 'day'], inplace=True, axis=1)

campaign.head()

C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9136\633018775.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['year'] = '2022'
C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9136\633018775.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['month'] = campaign['month'].replace(['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9136\63301877

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


## Part 3 : Saving the data

In [72]:
# exporting client to csv
client.to_csv('client.csv', index = False)

# exporting campaign to csv
campaign.to_csv('campaign.csv', index = False)

# exporting economics to csv
economics.to_csv('economics.csv', index = False)